In [30]:
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from urllib.request import urlopen
import json
import os
import datetime
import plotly.express as px
from statsmodels.tsa.arima.model import ARIMA
import math
from sklearn.metrics import mean_squared_error
from pmdarima.arima import auto_arima
import datetime
import matplotlib.dates as dates
import statsmodels.api as sm
from datetime import datetime
from pmdarima.arima import auto_arima
import statsmodels.api as sm

In [31]:
credentials = service_account.Credentials.from_service_account_file(
'chain-coders-5d47d58b2c33.json')

project_id = 'chain-coders'
client = bigquery.Client(credentials= credentials,project=project_id)

datasets = client.list_datasets()
for dataset in datasets:
  did = dataset.dataset_id
  # Optional to verify that you are able to access the datasets.
  # The datasets we care about are mimiciii_clinical and mimiciii_notes.
  print(did)

redfin


In [32]:
query = """SELECT * FROM `redfin.redfin-data-state` """
df = client.query(query).to_dataframe()

In [44]:
states = df.state_code.unique()
property_types = df.property_type_id.unique()
period_dates = df.period_end.unique()
df_pred = pd.DataFrame()
cols = ['state_code', 'property_type_id', 'period_end','property_type']
property_dict ={4:'Multi_Family' , -1 : "All_Residential" , 13:"Townhouse" , 3:"Condo", 6:"Single_Family_Residential"}

In [45]:
for state in states:
    for property_type in property_types:
        #print("Predicting for ", state, property_type)
        mask1 = df['state_code'] ==state
        mask2 = df['property_type_id'] == property_type

        df_out = df[mask1 & mask2]
        df_sar = df_out[['median_ppsf', 'period_end']]
        df_sar = df_sar.set_index('period_end')
        df_sar.index = pd.to_datetime(df_sar.index)
        ts = df_sar.resample('m').mean().fillna(method='ffill')
        if (ts.shape[0] < 50):
            print(" ###################. Not enough data for ", state, property_type)
            continue

        ts_train_ac=ts
        ts_train_bc=ts[ts.index[0]:( ts.index[ts.index.get_loc('2020-01-31', method='nearest')])]

        fit_ac = sm.tsa.statespace.SARIMAX(ts_train_ac.astype(float), order=(1, 1, 1),seasonal_order=(0,1,1,12)).fit(disp=False)
        fit_bc = sm.tsa.statespace.SARIMAX(ts_train_bc.astype(float), order=(1, 1, 1),seasonal_order=(0,1,1,12)).fit(disp=False)

        ts_predicted_bc = fit_bc.predict(start=ts_train_bc.index[0],end='2025-12-31')
        ts_predicted_ac = fit_ac.predict(start=ts_train_ac.index[0],end='2025-12-31')

        pred_bc = pd.Series(ts_predicted_bc, name='median_ppsf_without_covid').to_frame()
        pred_ac = pd.Series(ts_predicted_ac, name='median_ppsf_with_covid').to_frame()
        pred = pd.merge(pred_bc,pred_ac,  how='outer', left_index= True, right_index= True)

        
        pred['state_code'] = state
        pred['property_type_id'] = property_type
        pred['property_type'] = property_dict[property_type]

        pred = pred.reset_index().rename(columns={'index': 'period_end'})
        pred['period_end']=pred['period_end'].astype(str)
        pred['combined'] = pred[cols].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)

        if  df_pred.empty:
            df_pred = pred
        else:
            df_pred = df_pred.append(pred)

/usr/local/lib/python3.7/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/usr/local/lib/python3.7/site-packages/statsmodels/tsa/statespace/sarimax.py:1009: UserWarning: Non-invertible starting seasonal moving average Using zeros as starting parameters.
  warn('Non-invertible starting seasonal moving average'
/usr/local/lib/python3.7/site-packages/statsmodels/tsa/statespace/sarimax.py:1009: UserWarning: Non-invertible starting seasonal moving average Using zeros as starting parameters.
  warn('Non-invertible starting seasonal moving average'
/usr/local/lib/python3.7/site-packages/statsmodels/tsa/statespace/sarimax.py:868: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.
  ' zeros.' % warning_description)
/usr/local/lib/python3.7/site-packag

 ###################. Not enough data for  AK 13


/usr/local/lib/python3.7/site-packages/statsmodels/tsa/statespace/sarimax.py:1009: UserWarning: Non-invertible starting seasonal moving average Using zeros as starting parameters.
  warn('Non-invertible starting seasonal moving average'
/usr/local/lib/python3.7/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/usr/local/lib/python3.7/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/usr/local/lib/python3.7/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/usr/local/lib/python3.7/site-packages/statsmodels/tsa/statespace/sarimax

 ###################. Not enough data for  WV 3


In [46]:
df_orig = df.copy()
df_orig['combined'] = df_orig[cols].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)
df_orig = df_orig.set_index('combined')
df_pred= df_pred.set_index('combined')

#df_merged.reset_index()

In [47]:
df_merged = pd.merge(df_pred,df_orig,  how='outer', left_index= True, right_index= True, suffixes=('', '_y'))
df_merged.drop(df_merged.filter(regex='_y$').columns, axis=1, inplace=True)
df_merged["covid_impact"] = 100*((df_merged["median_ppsf_with_covid"]-df_merged["median_ppsf_without_covid"])/df_merged["median_ppsf_without_covid"])
df_merged = df_merged.reset_index()
df_merged = df_merged.drop('combined', axis=1)

In [48]:
job_config = bigquery.LoadJobConfig(

    # Optionally, set the write disposition. BigQuery appends loaded rows
    # to an existing table by default, but with WRITE_TRUNCATE write
    # disposition it replaces the table with the loaded data.
    write_disposition="WRITE_TRUNCATE",
)

In [49]:
table_id = "redfin.redfin-data-state-with-predictions3" 

In [50]:
job = client.load_table_from_dataframe(
    df_merged, table_id, job_config=job_config
)  # Make an API request.
job.result()  # Wait for the job to complete.

/usr/local/lib/python3.7/site-packages/google/cloud/bigquery/_pandas_helpers.py:554: UserWarning: Pyarrow could not determine the type of columns: city, parent_metro_region_metro_code.
  ", ".join(field.name for field in unknown_type_fields)


LoadJob<project=chain-coders, location=us-west2, id=8c16f9f2-4f6d-4fab-be0b-4841c0e12b26>